In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import glob
import cv2
from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout
import os
import seaborn as sns
from keras.applications.vgg16 import VGG16 

In [2]:
tf.__version__

'2.9.1'

In [3]:
SIZE=128
train_images = []
train_labels = [] 
for directory_path in glob.glob("dataset/training_set/*"):
    label = directory_path.split("\\")[-1]
    for img_path in glob.glob(os.path.join(directory_path, "*.png")): 
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        img = cv2.GaussianBlur(img, (3,3), 0)
        train_images.append(img)
        train_labels.append(label)
train_images = np.array(train_images)
train_labels = np.array(train_labels)

In [4]:
test_images = []
test_labels = [] 
for directory_path in glob.glob("dataset/test_set/*"):
    test_label = directory_path.split("\\")[-1]
    for img_path in glob.glob(os.path.join(directory_path, "*.png")):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        img = cv2.GaussianBlur(img, (3,3), 0)
        test_images.append(img)
        test_labels.append(test_label)
test_images = np.array(test_images)
test_labels = np.array(test_labels)

In [5]:
#Encode labels from text to integers.
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(test_labels)
test_labels_encoded = le.transform(test_labels)
le.fit(train_labels)
train_labels_encoded = le.transform(train_labels)

#Split data into test and train datasets 
x_train, y_train, x_test, y_test = train_images, train_labels_encoded, test_images, test_labels_encoded

In [6]:
# Normalize pixel values to between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0

#One hot encoding y values for neural network. 
from keras.utils import to_categorical
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)

# VGG 

In [7]:
#Load model without classifier/fully connected layers
VGG_model = VGG16(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3))
for layer in VGG_model.layers:
	layer.trainable = False
VGG_model.summary()  #Trainable parameters will be 0

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 128, 128, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 128, 128, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 64, 64, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 64, 64, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 64, 64, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 32, 32, 128)       0     

# Random Forest

In [ ]:
feature_extractor=VGG_model.predict(x_train)
features = feature_extractor.reshape(feature_extractor.shape[0], -1)

X_for_RF = features 
from sklearn.ensemble import RandomForestClassifier
RF_model = RandomForestClassifier(n_estimators = 100,random_state = 42) 

# Train the model on training data
RF_model.fit(X_for_RF, y_train)
#Send test data through same feature extractor process
X_test_feature = VGG_model.predict(x_test)
X_test_features = X_test_feature.reshape(X_test_feature.shape[0], -1)

#Predict using the trained RF model. 
prediction_RF = RF_model.predict(X_test_features)
prediction_RF = le.inverse_transform(prediction_RF)

from sklearn import metrics
print ("Accuracy = ", metrics.accuracy_score(test_labels, prediction_RF))
print("F1 score (micro) = ", metrics.f1_score(test_labels, prediction_RF, average='micro'))
print("F1 score (macro) = ", metrics.f1_score(test_labels, prediction_RF, average='macro'))
print("F1 score (weighted) = ", metrics.f1_score(test_labels, prediction_RF, average='weighted'))

print("Precision (micro) = ", metrics.precision_score(test_labels, prediction_RF, average='micro'))
print("Precision (macro) = ", metrics.precision_score(test_labels, prediction_RF, average='macro'))
print("Precision (weighted) = ", metrics.precision_score(test_labels, prediction_RF, average='weighted'))

print("Recall (micro) = ", metrics.recall_score(test_labels, prediction_RF, average='micro'))
print("Recall (macro) = ", metrics.recall_score(test_labels, prediction_RF, average='macro'))
print("Recall (weighted) = ", metrics.recall_score(test_labels, prediction_RF, average='weighted'))

print("Confusion Matrix = ",metrics.confusion_matrix(test_labels, prediction_RF))

# XGBoost

In [ ]:
pip install xgboost  

In [ ]:
feature_extractor=VGG_model.predict(x_train)
features = feature_extractor.reshape(feature_extractor.shape[0], -1)
X_for_training = features

import xgboost as xgb
model = xgb.XGBClassifier()
model.fit(X_for_training, y_train) 

#Send test data through same feature extractor process
X_test_feature = VGG_model.predict(x_test)
X_test_features = X_test_feature.reshape(X_test_feature.shape[0], -1)

#Predict using the trained RF model. 
prediction = model.predict(X_test_features) 
prediction = le.inverse_transform(prediction)

from sklearn import metrics
print ("Accuracy = ", metrics.accuracy_score(test_labels, prediction))
print("F1 score (micro) = ", metrics.f1_score(test_labels, prediction, average='micro'))
print("F1 score (macro) = ", metrics.f1_score(test_labels, prediction, average='macro'))
print("F1 score (weighted) = ", metrics.f1_score(test_labels, prediction, average='weighted'))

print("Precision (micro) = ", metrics.precision_score(test_labels, prediction, average='micro'))
print("Precision (macro) = ", metrics.precision_score(test_labels, prediction, average='macro'))
print("Precision (weighted) = ", metrics.precision_score(test_labels, prediction, average='weighted'))

print("Recall (micro) = ", metrics.recall_score(test_labels, prediction, average='micro'))
print("Recall (macro) = ", metrics.recall_score(test_labels, prediction, average='macro'))
print("Recall (weighted) = ", metrics.recall_score(test_labels, prediction, average='weighted'))
print("Confusion Matrix = ",metrics.confusion_matrix(test_labels, prediction))